In [42]:
import pandas as pd
import time
from multiprocessing import Pool, cpu_count

In [43]:
df1 = pd.read_csv('larger_fees.csv')
df2 = pd.read_csv('larger_students.csv')

In [ ]:

def most_frequent_payment_day(group):
    return group['day_of_month'].mode()[0] if not group['day_of_month'].mode().empty else None

In [ ]:
def parallel_task(student_group):
    student_id, group = student_group
    most_consistent_day = most_frequent_payment_day(group)
    return student_id, most_consistent_day

# Prepare data for parallel processing
student_groups = list(df1.groupby('student_id'))

# Start Pool before timing
with Pool(cpu_count()) as pool:
    start_parallel = time.time()
    parallel_results = pool.map(parallel_task, student_groups)
    end_parallel = time.time()

# Convert results to DataFrame and Merge with df2
most_consistent_parallel = pd.DataFrame(parallel_results, columns=['student_id', 'Most Consistent Payment Day'])
result_parallel = pd.merge(df2, most_consistent_parallel, on='student_id', how='left')

# Compute and print duration
parallel_duration = end_parallel - start_parallel
print(f"Parallel Processing Time: {parallel_duration:.4f} seconds")

Parallel Processing Time: 0.0709 seconds


In [ ]:
result_parallel.head()

,student_id,name,department,semester,Most Consistent Payment Day
0,72,Lisa Singh,Computer Science,4,12
1,41,Emily Burton,Computer Science,8,29
2,57,Cassandra Patterson,Electrical Engineering,6,12
3,75,Ashley Thomas,Software Engineering,8,12
4,18,Joseph Sawyer,Mechanical Engineering,1,29


In [55]:
serial_duration = 0.0906
parallel_duration = 0.0709
# Percentage improvement
percent_faster = round(((serial_duration - parallel_duration) / serial_duration) * 100, 2)
print(f"Parallel Processing is {percent_faster}% faster than Serial Processing in my case.")

Parallel Processing is 21.74% faster than Serial Processing in my case.
